In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/properatti.csv')
data.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [3]:
''' Vemos las columnas '''

data.loc[10,:]


Unnamed: 0                                                                   10
operation                                                                  sell
property_type                                                             house
place_name                                                              Córdoba
place_with_parent_names                             |Argentina|Córdoba|Córdoba|
country_name                                                          Argentina
state_name                                                              Córdoba
geonames_id                                                         3.86026e+06
lat-lon                                                 -31.4200833,-64.1887761
lat                                                                    -31.4201
lon                                                                    -64.1888
price                                                                     70000
currency                                

In [4]:
''' vemos los nulos '''

data['price_aprox_usd'].isna().sum()


20410

In [5]:
''' Exploramos las columnas de los precios para ver si hay datos no nulos'''

data.loc[data['price_aprox_usd'].isna()].loc[:,['price','price_aprox_local_currency','price_usd_per_m2','price_per_m2']]




,price,price_aprox_local_currency,price_usd_per_m2,price_per_m2
5,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN
41,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN
...,...,...,...,...
121210,NaN,NaN,NaN,NaN
121211,NaN,NaN,NaN,NaN
121212,NaN,NaN,NaN,NaN
121213,NaN,NaN,NaN,NaN


In [7]:
''' quitar los nulos precios '''

data = data.drop(data[data['price_aprox_usd'].isna()].index)
cant_nulls = data.isnull().sum()
print(cant_nulls)



Unnamed: 0                        0
operation                         0
property_type                     0
place_name                       23
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         61231
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [8]:
''' completar los places names '''
data.loc[data.place_name.isnull(),'place_name'] = data.loc[data.place_name.isnull()].place_with_parent_names.str.split('|', expand=True)[3]
cant_nulls = data.isnull().sum()
print(cant_nulls)


Unnamed: 0                        0
operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         61231
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [9]:
''' Leo los ambientes para completar la info'''

import re

patron_ambientes = re.compile("(?P<Ambientes>\d+)(?P<tipo>amb|\samb)",flags = re.IGNORECASE)
patron_habitaciones = re.compile("(?P<Habitaciones>\d+)(?P<tipo>hab|\shab|dorm|\sdorm)",flags = re.IGNORECASE)

def extractRooms(row):
    ambientes_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        ambientes_match = patron_ambientes.search(row['description'])
    if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
        ambientes_match = patron_ambientes.search(row['title'])
    if (ambientes_match is not None):
        return int(ambientes_match.groups()[0]) 
    else: 
        if (row['description'] and isinstance(row['description'],str)):
            ambientes_match = patron_habitaciones.search(row['description'])
        if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
            ambientes_match = patron_habitaciones.search(row['title'])
        if (ambientes_match is not None):
            return int(ambientes_match.groups()[0])+1
        return None
        
        
data.loc[data.rooms.isnull(),'rooms'] = data.loc[data.rooms.isnull()].apply(extractRooms,axis=1)


In [ ]:
cant_nulls = data.isnull().sum()
print(cant_nulls)

In [ ]:

patron_meters = re.compile("(?P<Metros>\d+)(?P<tipo>metro|\smetro|mts|\smts|m2|\sm2)",flags = re.IGNORECASE)


def extractMeters(row):
    meters_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        meters_match = patron_meters.search(row['description'])
    if ((patron_meters is None) and row['title'] and isinstance(row['title'],str)):
        meters_match = patron_meters.search(row['title'])
    if (meters_match is not None):
        return meters_match.groups()[0]
    return None

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].apply(extractMeters,axis=1)

data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull()].apply(lambda x: x['surface_covered_in_m2'],axis=1)




In [10]:
cant_nulls = data.isnull().sum()
print(cant_nulls)

Unnamed: 0                        0
operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         24521
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [11]:
pd.options.display.float_format = '{:,.2f}'.format

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].surface_total_in_m2.fillna(value=np.nan)
data.loc[:,'surface_total_in_m2'] = data.loc[:,'surface_total_in_m2'].astype("float")




In [16]:
data.price.describe()

count       100,810.00
mean        468,525.93
std       2,260,100.59
min               0.00
25%         110,000.00
50%         185,000.00
75%         420,000.00
max     650,000,000.00
Name: price, dtype: float64